<a href="https://colab.research.google.com/github/C-Gibson20/financial-sentiment-analysis/blob/main/Financial_Sentiment_Analysis_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports...

In [ ]:
!pip install datasets evaluate nlpaug

In [ ]:
from datasets import Dataset as HFDataset
from evaluate import load
import pandas as pd
import nlpaug.augmenter.word as naw
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    pipeline,
    EarlyStoppingCallback,
    TrainerCallback,
    AdamW,
    get_linear_schedule_with_warmup)

# Dataset Preparation


### Kaggle Dataset Download

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/APIs/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download sbhatti/financial-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis
License(s): CC0-1.0
100% 276k/276k [00:00<00:00, 559kB/s]
100% 276k/276k [00:00<00:00, 559kB/s]


In [ ]:
! unzip financial-sentiment-analysis.zip

Archive:  financial-sentiment-analysis.zip
  inflating: data.csv                


### Data Loading and Preprocessing

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
def encode(sentences, sentiments, tokenizer):
    encodings = tokenizer(sentences, truncation=True, padding=True, return_tensors='pt')
    labels = torch.tensor(sentiments)
    encodings['labels'] = labels
    return encodings

def data_loader(encodings, labels, batch_size=32, shuffle=False):
    dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def augment_sentences(sentences, sentiments, augment_rate=0.2):
    aug_sentences, aug_sentiments = [], []
    for sentence, sentiment in zip(sentences, sentiments):
        aug_sentences.append(sentence)
        aug_sentiments.append(sentiment)

        if random.random() < augment_rate:
            aug_sentence = synonym_aug.augment(sentence)

            if isinstance(aug_sentence, list):
                aug_sentence = ' '.join(aug_sentence)

            aug_sentences.append(aug_sentence)
            aug_sentiments.append(sentiment)

    return aug_sentences, aug_sentiments

In [ ]:
sentiment_encoding = {'negative': 0, 'neutral': 1, 'positive': 2}
financial_sentiment_df = pd.read_csv('data.csv')
financial_sentiment_df['Sentiment'] = financial_sentiment_df['Sentiment'].map(sentiment_encoding)

sentences = financial_sentiment_df['Sentence'].tolist()
sentiments = financial_sentiment_df['Sentiment'].tolist()

train_sentences, val_test_sentences, train_sentiments, val_test_sentiments = train_test_split(sentences, sentiments, test_size=0.2, random_state=42, stratify=sentiments)
val_sentences, test_sentences, val_sentiments, test_sentiments = train_test_split(val_test_sentences, val_test_sentiments, test_size=0.5, random_state=42, stratify=val_test_sentiments)

synonym_aug = naw.SynonymAug(aug_src='wordnet', aug_max=2)
aug_train_sentences, aug_train_sentiments = augment_sentences(train_sentences, train_sentiments)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
train_encodings = encode(aug_train_sentences, aug_train_sentiments, tokenizer)
val_encodings = encode(val_sentences, val_sentiments, tokenizer)
test_encodings = encode(test_sentences, test_sentiments, tokenizer)

train_loader = data_loader(train_encodings, train_encodings['labels'], shuffle=True)
val_loader = data_loader(val_encodings, val_encodings['labels'])
test_loader = data_loader(test_encodings, test_encodings['labels'])

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_sentiments), y=train_sentiments)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda')

# Model Training and Initialisation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def create_optimizer_with_warmup(model, num_warmup_steps=500, total_steps=5000, learning_rate=2e-5, dft_rate=1.2):
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = []

    for i in range(12):
        layer_decay = {
            'params': [p for n, p in model.bert.encoder.layer[i].named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': 0.01,
            'lr': learning_rate / (dft_rate ** (11 - i))
        }
        layer_no_decay = {
            'params': [p for n, p in model.bert.encoder.layer[i].named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0,
            'lr': learning_rate / (dft_rate ** (11 - i))
        }
        optimizer_grouped_parameters.append(layer_decay)
        optimizer_grouped_parameters.append(layer_no_decay)

    classifier_params = {
        'params': model.classifier.parameters(),
        'lr': learning_rate,
        'weight_decay': 0.01
    }

    optimizer_grouped_parameters.append(classifier_params)
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)

    return optimizer, scheduler

In [ ]:
class GradualUnfreezingCallback(TrainerCallback):
    def __init__(self, model, freeze_start_layer=11):
        self.model = model
        self.currently_frozen_layer = freeze_start_layer

    def on_epoch_begin(self, args, state, control, **kwargs):
        if self.currently_frozen_layer >= 0:
            for param in self.model.bert.encoder.layer[self.currently_frozen_layer].parameters():
                param.requires_grad = True
            print(f"Unfreezing layer {self.currently_frozen_layer}")
            self.currently_frozen_layer -= 1

for param in model.bert.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, class_weights=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.class_weights = class_weights

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(weight=self.class_weights)(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss

In [ ]:
class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = FocalLoss(class_weights=self.class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(eval_pred):
    metric_accuracy = load('accuracy')
    metric_f1 = load('f1')
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric_accuracy.compute(predictions=predictions, references=labels)
    f1 = metric_f1.compute(predictions=predictions, references=labels, average='weighted')
    return {'accuracy': accuracy['accuracy'], 'f1': f1['f1']}

In [ ]:
optimizer, scheduler = create_optimizer_with_warmup(model)
gradual_unfreezing_callback = GradualUnfreezingCallback(model)
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01)

train_dataset = HFDataset.from_dict(train_encodings)
val_dataset = HFDataset.from_dict(val_encodings)
test_dataset = HFDataset.from_dict(test_encodings)

In [ ]:
training_arguments = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.001,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    warmup_steps=500,
)

trainer = WeightedTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler),
    callbacks=[gradual_unfreezing_callback, early_stopping_callback],
    class_weights=class_weights
)

In [ ]:
trainer.train()

Unfreezing layer 11


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.458430,0.583904,0.572282
2,No log,0.253648,0.635274,0.648155
3,0.355000,0.135426,0.720890,0.739324
4,0.355000,0.116202,0.746575,0.761829
5,0.355000,0.124441,0.756849,0.770158
6,0.069800,0.127756,0.758562,0.772911
7,0.069800,0.132056,0.756849,0.770886


Unfreezing layer 10
Unfreezing layer 9
Unfreezing layer 8
Unfreezing layer 7
Unfreezing layer 6
Unfreezing layer 5


TrainOutput(global_step=1239, training_loss=0.17799699104438393, metrics={'train_runtime': 462.1837, 'train_samples_per_second': 122.159, 'train_steps_per_second': 3.83, 'total_flos': 1990389282331464.0, 'train_loss': 0.17799699104438393, 'epoch': 7.0})

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.08022227138280869,
 'eval_accuracy': 0.7965811965811965,
 'eval_f1': 0.8099998432077777,
 'eval_runtime': 6.0054,
 'eval_samples_per_second': 97.413,
 'eval_steps_per_second': 3.164,
 'epoch': 7.0}

In [ ]:
saved_model_name = 'finetuned_bert_with_discriminative_ft'
model.save_pretrained(saved_model_name)
tokenizer.save_pretrained(saved_model_name)

('finetuned_bert_with_discriminative_ft/tokenizer_config.json',
 'finetuned_bert_with_discriminative_ft/special_tokens_map.json',
 'finetuned_bert_with_discriminative_ft/vocab.txt',
 'finetuned_bert_with_discriminative_ft/added_tokens.json')

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)

sentence = "The market is performing poorly today."
result = sentiment_pipeline(sentence)
print(result)

[{'label': 'LABEL_0', 'score': 0.7712957262992859}]
